In [ ]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00


In [ ]:
!unzip /content/data.zip

unzip:  cannot find or open /content/data.zip, /content/data.zip.zip or /content/data.zip.ZIP.


In [ ]:
!pip install arabert
!pip install datasets
!pip install evaluate
!pip install accelerate==0.21.0
!pip install transformers==4.30.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186460 sha256=b3027a20945f5aacd55aeed3523364ed63a3af21636bc05f3db94293e7595701
  Stored in directory: /root/.cache/pip/wheels/10/f0/fd/4813b1177405693e8da9cdea839f0fb64fde161380e058c827
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninst

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/data.zip /content/data.zip
!unzip /content/data.zip

Archive:  /content/data.zip
   creating: data/
  inflating: data/Arabic-SQuAD.json  
  inflating: data/arcd-test.json     
  inflating: data/arcd-train.json    
  inflating: data/MLQA-dev-context-ar-question-ar.json  
  inflating: data/MLQA-test-context-ar-question-ar.json  
  inflating: data/tydiqa-goldp-dev-arabic.json  
  inflating: data/tydiqa-goldp-v1.1-train-ar.json  


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

# Specify the path to your JSON file
json_file_path = '/content/data/Arabic-SQuAD.json'

# Load the dataset
dataset = load_dataset('json', data_files=json_file_path)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['data', 'version'],
        num_rows: 1
    })
})

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset

train_data = dataset['train']['data'][0]

def transform_data(data):
    transformed_data = {
        'id': [],
        'title': [],
        'context': [],
        'question': [],
        'answers': [],
    }

    for article in data:
        title = article['title']
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                transformed_data['id'].append(qa['id'])
                transformed_data['title'].append(title)
                transformed_data['context'].append(context)
                transformed_data['question'].append(qa['question'])
                transformed_data['answers'].append(qa['answers'])

    return transformed_data


transformed_train_data = transform_data(train_data)


transformed_dataset = Dataset.from_dict(transformed_train_data)


transformed_dataset_dict = DatasetDict({
    'train': transformed_dataset
})

print(transformed_dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 48344
    })
})


In [ ]:
squad = transformed_dataset.train_test_split(test_size=0.2)

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA")
model = AutoModelForQuestionAnswering.from_pretrained("ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/851 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/538M [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        # Assuming each answer in answers is a list of answer dicts
        answer = answers[i][0]  # Take the first answer
        start_char = answer["answer_start"]
        end_char = start_char + len(answer["text"])

        sequence_ids = inputs.sequence_ids(i)

        # Initialization to handle cases where context is empty or no answer is found
        context_start, context_end = 0, len(offsets) - 1

        # Find start and end of context within the offsets
        try:
            context_start = next(idx for idx, (start, end) in enumerate(offsets) if start is not None and sequence_ids[idx] == 1)
            context_end = next(idx for idx, (start, end) in enumerate(reversed(offsets), 1) if start is not None and sequence_ids[-idx] == 1)
            context_end = len(offsets) - context_end  # Adjust index because of reverse
        except StopIteration:
            # Handle case where context_start or context_end are not found
            pass

        # Adjust start_positions and end_positions
        start_position = 0
        end_position = 0
        for idx, (start, end) in enumerate(offsets[context_start:context_end+1], context_start):
            if start <= start_char < end:
                start_position = idx
            if start < end_char <= end:
                end_position = idx
                break

        start_positions.append(start_position)
        end_positions.append(end_position)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/38675 [00:00<?, ? examples/s]

Map:   0%|          | 0/9669 [00:00<?, ? examples/s]

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
tokenized_squad

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 38675
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 9669
    })
})

In [ ]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="CUTD/qnAr",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    logging_steps=10,
    save_steps=5000,
    warmup_steps=500,
    fp16=True,
    gradient_accumulation_steps=4,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/CUTD/qnAr into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,1.879700,1.884302
1,1.756200,1.887922
2,1.665900,1.932394


TrainOutput(global_step=3624, training_loss=1.7570587675566178, metrics={'train_runtime': 883.899, 'train_samples_per_second': 131.265, 'train_steps_per_second': 4.1, 'total_flos': 2.2724586988194816e+16, 'train_loss': 1.7570587675566178, 'epoch': 3.0})

In [ ]:
# Now to save the model
trainer.save_model("CUTD/qnAr")

In [ ]:
!zip -r CUTD/qnAr.zip CUTD/qnAr

  adding: CUTD/qnAr/ (stored 0%)
  adding: CUTD/qnAr/tokenizer_config.json (deflated 42%)
  adding: CUTD/qnAr/.gitignore (stored 0%)
  adding: CUTD/qnAr/tokenizer.json (deflated 74%)
  adding: CUTD/qnAr/training_args.bin (deflated 50%)
  adding: CUTD/qnAr/README.md (deflated 49%)
  adding: CUTD/qnAr/vocab.txt (deflated 65%)
  adding: CUTD/qnAr/.git/ (stored 0%)
  adding: CUTD/qnAr/.git/lfs/ (stored 0%)
  adding: CUTD/qnAr/.git/lfs/objects/ (stored 0%)
  adding: CUTD/qnAr/.git/lfs/objects/a7/ (stored 0%)
  adding: CUTD/qnAr/.git/lfs/objects/a7/ac/ (stored 0%)
  adding: CUTD/qnAr/.git/lfs/objects/a7/ac/a7ac6f9b25e2532a4a6ec150c120fd08df14a35c30c63ae5fef56afb8683d4b7 (deflated 69%)
  adding: CUTD/qnAr/.git/lfs/objects/ed/ (stored 0%)
  adding: CUTD/qnAr/.git/lfs/objects/ed/17/ (stored 0%)
  adding: CUTD/qnAr/.git/lfs/objects/ed/17/ed1739870151258c7a19b2e7facd0508f7787b9a9b9004f106f2db975d2d4487 (deflated 50%)
  adding: CUTD/qnAr/.git/lfs/objects/d2/ (stored 0%)
  adding: CUTD/qnAr/.git/lf

In [ ]:
!cp CUTD/qnAr.zip /content/drive/MyDrive/

In [ ]:
question = "في أي المجالات يتم تطبيق الذكاء الاصطناعي؟"
context = "الذكاء الاصطناعي هو فرع من فروع علم الكمبيوتر يعمل على تطوير الآلات والبرمجيات بقدرات تحاكي الذكاء البشري. يشمل ذلك التعلم، الاستنتاج، التعرف على الأنماط، فهم اللغة الطبيعية، والرؤية الحاسوبية. يتم تطبيق الذكاء الاصطناعي في العديد من المجالات مثل الطب، الهندسة، التسويق، والألعاب."

In [ ]:
from transformers import pipeline

pipe = pipeline("question-answering", model="CUTD/qnAr")
pipe(question=question, context=context)['answer']

'الطب، الهندسة، التسويق، والألعاب'